In [1]:
#Python libraries

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt, geometry, shape, box
import osmnx as ox
import json
import folium
import networkx as nx
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex
import numpy as np

In [ ]:
#The code processes spatial data to analyze buildings and services within selected parishes and a 1.5 km buffer around them, filtering and converting geometries for further spatial analysis

In [ ]:
# Function to load CSV data and convert WKT to geometry
def load_csv_with_geometry(path, geom_column='geometry_wkt'):
    df = pd.read_csv(path, dtype=str, low_memory=False)
    if geom_column in df.columns:
        df['geometry'] = df[geom_column].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry='geometry')
    return df

# Safe function to convert a GeoJSON string into a shapely geometry
def safe_geojson_loads(geojson_str):
    try:
        return shape(json.loads(geojson_str))
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# Paths to relevant files
path_edificios = 'edificios.csv'
path_servicos = 'servicos.csv'
path_freguesias = 'freguesia.csv'

# Load data
edificios = load_csv_with_geometry(path_edificios, geom_column='geometry_wkt')
servicos = pd.read_csv(path_servicos, dtype=str, low_memory=False)
freguesias = pd.read_csv(path_freguesias, sep=';', encoding='latin1')

# Convert GeoJSON representations into geometric objects
freguesias['geometry'] = freguesias['Geo Shape'].apply(safe_geojson_loads)
gdf_freguesias = gpd.GeoDataFrame(freguesias, geometry='geometry').dropna(subset=['geometry'])

# List of parishes to be processed
freguesias_nomes = ['Aldoar', 'Nevogilde', 'Foz do Douro']

# Filter the selected parishes
freguesias_selecionadas = gdf_freguesias[gdf_freguesias['Official Name Parish'].str.contains('|'.join(freguesias_nomes), case=False, na=False)]

# Check if parishes were found
if freguesias_selecionadas.empty:
    print("None of the selected parishes were found!")
else:
    print("Selected parishes found.")
    freguesias_geom = freguesias_selecionadas.unary_union  # Merge the geometries of the selected parishes

# Convert DataFrames to GeoDataFrames if they are not already
if not isinstance(edificios, gpd.GeoDataFrame):
    edificios = gpd.GeoDataFrame(edificios, geometry='geometry')

# Filter buildings within the selected parishes
edificios_selecionadas = edificios[edificios.geometry.within(freguesias_geom)]

# Adjust column names correctly with uppercase letters
servicos['geometry'] = servicos.apply(lambda row: wkt.loads(f"POINT ({row['Longitude']} {row['Latitude']})"), axis=1)

# Create a 1.5 km buffer around the parish geometries
buffer_1_5km = freguesias_geom.buffer(1500)

# Filter services that are within the buffer (both inside and outside the parishes)
servicos_buffer_1_5km = servicos[gpd.GeoSeries(servicos['geometry']).within(buffer_1_5km)]

# Convert Polygons and MultiPolygons to Points (centroids)
servicos_buffer_1_5km.loc[:, 'geometry'] = servicos_buffer_1_5km['geometry'].apply(
    lambda geom: geom.centroid if geom.geom_type in ['Polygon', 'MultiPolygon'] else geom
)

# Convert to GeoDataFrames
servicos_gdf_selecionadas = gpd.GeoDataFrame(servicos_buffer_1_5km, geometry='geometry')
edificios_gdf_selecionadas = gpd.GeoDataFrame(edificios_selecionadas, geometry='geometry')

# Create a road network graph for the selected parishes
G_total = ox.graph_from_polygon(freguesias_geom, network_type='walk')

# Print some information to verify results
print(f"Number of buildings in the selected parishes: {len(edificios_gdf_selecionadas)}")
print(f"Number of services within the 1.5 km buffer: {len(servicos_gdf_selecionadas)}")

In [ ]:
#The code creates an interactive Folium map displaying the walkable road network of selected parishes, using OpenStreetMap data

In [ ]:
# Define the center of the map based on the combined geometry of the selected parishes
centro = [freguesias_geom.centroid.y, freguesias_geom.centroid.x]
m = folium.Map(location=centro, zoom_start=14)

# Add the road network to the map
for u, v, data in G_total.edges(data=True):
    if 'geometry' in data:
        # Add the geometry if available
        folium.PolyLine([(coord[1], coord[0]) for coord in data['geometry'].coords],
                        color="blue", weight=2.5, opacity=0.8).add_to(m)
    else:
        # If there is no geometry, draw a simple line between the nodes
        x_u, y_u = G_total.nodes[u]['x'], G_total.nodes[u]['y']
        x_v, y_v = G_total.nodes[v]['x'], G_total.nodes[v]['y']
        folium.PolyLine([(y_u, x_u), (y_v, x_v)], color="blue", weight=2.5, opacity=0.8).add_to(m)

# Display the map
m

In [ ]:
#The code analyzes the northern area of selected parishes by calculating the average walking distance from buildings to nearby services using the street network

In [ ]:
# Calculate the bounding box of the selected parishes area
minx, miny, maxx, maxy = freguesias_geom.bounds

# Split the area into two parts (north/south)
mid_y = (maxy + miny) / 2

# Create the geometry for the northern area
norte_geom = freguesias_geom.intersection(box(minx, mid_y, maxx, maxy))

# Filter buildings and services located in the northern area of the selected parishes
edificios_norte = edificios_gdf_selecionadas[edificios_gdf_selecionadas.geometry.within(norte_geom)].copy()
servicos_norte = servicos_gdf_selecionadas[servicos_gdf_selecionadas.geometry.within(norte_geom)].copy()

# Function to find the nearest node in the graph
def encontrar_no_mais_proximo(G, ponto):
    if ponto.geom_type == 'Point':
        return ox.distance.nearest_nodes(G, X=ponto.x, Y=ponto.y)
    elif ponto.geom_type in ['Polygon', 'MultiPolygon']:
        centroid = ponto.centroid
        return ox.distance.nearest_nodes(G, X=centroid.x, Y=centroid.y)
    else:
        raise ValueError(f"Unsupported geometry type: {ponto.geom_type}")

# Add nearest nodes for buildings and services
edificios_norte['nearest_node'] = edificios_norte['geometry'].apply(lambda geom: encontrar_no_mais_proximo(G_total, geom))
servicos_norte['nearest_node'] = servicos_norte['geometry'].apply(lambda geom: encontrar_no_mais_proximo(G_total, geom))

# Function to calculate the average distance to services within a 1.5 km radius
def calcular_distancia_media(G, nodo_edificio, servicos_gdf, distancia_max=1500):
    distancias = []
    for nodo_servico in servicos_gdf['nearest_node']:
        try:
            distancia = nx.shortest_path_length(G, nodo_edificio, nodo_servico, weight='length')
            if distancia <= distancia_max:
                distancias.append(distancia)
        except nx.NetworkXNoPath:
            continue
    
    if distancias:
        return sum(distancias) / len(distancias)  # Average distance
    else:
        return None  # If no services exist within a 1.5 km radius

# Function to apply parallelization to compute average distances
def calcular_distancias_medias_para_todos(G, edificios_gdf, servicos_gdf, distancia_max=1500):
    return Parallel(n_jobs=-1)(
        delayed(calcular_distancia_media)(G, nodo_edificio, servicos_gdf, distancia_max)
        for nodo_edificio in edificios_gdf['nearest_node']
    )

# Compute the average distances for all buildings in the northern area of the selected parishes
edificios_norte['distancia_media_servicos'] = calcular_distancias_medias_para_todos(
    G_total, edificios_norte, servicos_norte, distancia_max=1500
)

# Display results for the northern area
print("Results for the Northern Area of the Selected Parishes:")
print(edificios_norte[['geometry', 'distancia_media_servicos']].head())

In [ ]:
#The code analyzes the southern area of selected parishes by calculating the average walking distance from buildings to nearby services using the street network

In [ ]:
# Create the geometry for the southern area of the selected parishes
sul_geom = freguesias_geom.intersection(box(minx, miny, maxx, mid_y))

# Filter buildings and services located in the southern area of the selected parishes
edificios_sul = edificios_gdf_selecionadas[edificios_gdf_selecionadas.geometry.within(sul_geom)].copy()
servicos_sul = servicos_gdf_selecionadas[servicos_gdf_selecionadas.geometry.within(sul_geom)].copy()

# Add nearest nodes for buildings and services
edificios_sul.loc[:, 'nearest_node'] = edificios_sul['geometry'].apply(lambda geom: encontrar_no_mais_proximo(G_total, geom))
servicos_sul.loc[:, 'nearest_node'] = servicos_sul['geometry'].apply(lambda geom: encontrar_no_mais_proximo(G_total, geom))

# Compute the average distances for all buildings in the southern area of the selected parishes
edificios_sul.loc[:, 'distancia_media_servicos'] = calcular_distancias_medias_para_todos(
    G_total, edificios_sul, servicos_sul, distancia_max=1500
)

# Display results for the southern area
print("Results for the Southern Area of the Selected Parishes:")
print(edificios_sul[['geometry', 'distancia_media_servicos']].head())

In [ ]:
#The code merges the results from the northern and southern areas of the selected parishes, saves them to a CSV file, and displays a sample for verification

In [ ]:
# Merge the results of the northern and southern areas of the selected parishes
resultados_unidos = pd.concat([edificios_norte, edificios_sul])

# Save the merged results to a CSV file
resultados_unidos.to_csv('resultados_distancia_media_servicos_aldoar_global.csv', index=False)

# Display a sample of the merged results for verification
print("Unified Results for the Selected Parishes:")
print(resultados_unidos[['geometry', 'distancia_media_servicos']].head())

In [ ]:
#

In [ ]:
# Check if the 'geometry_wkt' column is present and convert it to Shapely geometries
if 'geometry_wkt' in resultados_unidos.columns:
    resultados_unidos['geometry'] = resultados_unidos['geometry_wkt'].apply(wkt.loads)

# Create a GeoDataFrame from the merged results
resultados_unidos = gpd.GeoDataFrame(resultados_unidos, geometry='geometry')

# Set the CRS to WGS 84 (EPSG:4326) if the coordinates are in lat/lon
resultados_unidos = resultados_unidos.set_crs(epsg=4326)

# Display a sample to verify
print("Merged Results as GeoDataFrame for the Selected Parishes:")
print(resultados_unidos.head())

In [ ]:
#The code creates an interactive map showing buildings' average distance to services with color coding and service markers

In [ ]:
# Ensure the GeoDataFrame is in CRS EPSG:4326
resultados_unidos = gpd.GeoDataFrame(resultados_unidos, geometry='geometry', crs="EPSG:4326")

# Reproject to a projected CRS (e.g., EPSG:3857) to correctly calculate centroids
resultados_unidos_proj = resultados_unidos.to_crs(epsg=3857)

# Calculate the map center based on the centroids of the reprojected geometries
centro_proj = resultados_unidos_proj.geometry.centroid
centro_lat = centro_proj.to_crs(epsg=4326).y.mean()
centro_lon = centro_proj.to_crs(epsg=4326).x.mean()

# Create the map centered on the calculated center
m = folium.Map(location=[centro_lat, centro_lon], zoom_start=14)

# Determine the color based on the average distance to services
def get_color(distancia_media_servicos):
    cmap = plt.get_cmap('YlOrRd')
    norm = plt.Normalize(vmin=resultados_unidos['distancia_media_servicos'].min(), vmax=resultados_unidos['distancia_media_servicos'].max())
    return to_hex(cmap(norm(distancia_media_servicos)))

# Create a layer for buildings
edificios_layer = folium.FeatureGroup(name='Buildings').add_to(m)

# Add buildings to the map
for idx, row in resultados_unidos.iterrows():
    folium.GeoJson(
        data=row.geometry.__geo_interface__,
        style_function=lambda feature, color=get_color(row['distancia_media_servicos']): {
            'fillColor': color,
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=f'Average distance to services: {row.distancia_media_servicos:.2f} meters'
    ).add_to(edificios_layer)

# Create a layer for services
servicos_layer = folium.FeatureGroup(name='Services', show=False).add_to(m)

# Add services to the map
for idx, row in servicos_gdf_selecionadas.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['Name'],
        icon=folium.Icon(color='green')
    ).add_to(servicos_layer)

# Add layer control
folium.LayerControl().add_to(m)

# Add legend to the map
colormap = plt.get_cmap('YlOrRd')
norm = plt.Normalize(vmin=resultados_unidos['distancia_media_servicos'].min(), vmax=resultados_unidos['distancia_media_servicos'].max())
color_scale = folium.StepColormap(
    [to_hex(colormap(norm(i))) for i in np.linspace(resultados_unidos['distancia_media_servicos'].min(), resultados_unidos['distancia_media_servicos'].max(), 256)],
    vmin=resultados_unidos['distancia_media_servicos'].min(),
    vmax=resultados_unidos['distancia_media_servicos'].max(),
    caption='Average Distance to Services (meters)'
)
color_scale.add_to(m)

# Display the map (if using Jupyter Notebook)
m